In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import f_oneway

def plot_features_cat_regression(dataframe, target_col="", columns=[], pvalue=0.05, with_individual_plot=False):
    """
    Visualiza la relación entre variables categóricas y una variable objetivo numérica mediante histogramas agrupados.

    Argumentos:
    dataframe (pd.DataFrame): DataFrame que contiene los datos.
    target_col (str): Nombre de la columna objetivo. Debe ser numérica.
    columns (list): Lista de strings de columnas categóricas a analizar. Por defecto es una lista vacía.
    pvalue (float): Por defecto es 0.05.
    with_individual_plot (bool): Por defecto es False, y no se generan gráficos. Si es True, genera gráficos individuales para cada variable categórica seleccionada.

    Devuelve:
    list: Lista de columnas categóricas que tienen una relación significativa con la columna objetivo.
    """
    # Verificación
    if not isinstance(dataframe, pd.DataFrame): #comprobar si un objeto pertenece a una clase o tipo específico
        raise ValueError("El argumento 'dataframe' debe ser un DataFrame de pandas.")

    if not isinstance(target_col, str) or target_col == "":
        raise ValueError("El argumento 'target_col' debe ser una cadena no vacía.")

    if target_col not in dataframe.columns:
        raise ValueError(f"La columna objetivo '{target_col}' no está en el DataFrame.")

    if not pd.api.types.is_numeric_dtype(dataframe[target_col]):
        raise ValueError(f"La columna objetivo '{target_col}' debe ser de tipo numérico.")

    if not isinstance(columns, list):
        raise ValueError("El argumento 'columns' debe ser una lista.")

    if not columns:
        columns = dataframe.select_dtypes(include=["object", "category"]).columns.tolist()

    significant_columns = []

    for col in columns:
        if col not in dataframe.columns:
            continue

        if not pd.api.types.is_categorical_dtype(dataframe[col]) and not pd.api.types.is_object_dtype(dataframe[col]):
            continue #Verifica si la columna col es categorica o de tipo object

        # Prueba ANOVA para verificar relación significativa
        groups = [dataframe[dataframe[col] == category][target_col] for category in dataframe[col].dropna().unique()]
        if len(groups) > 1:
            stat, p = f_oneway(*groups) #descompone la lista groups en elementos individuales, pasando cada grupo como un argumento separado.
            if p < pvalue:
                significant_columns.append(col)

                # Generar gráfico si se solicita
                if with_individual_plot:
                    plt.figure(figsize=(10, 6))
                    sns.boxplot(x=col, y=target_col, data=dataframe)
                    plt.title(f"Relación entre {col} y {target_col} (p-value: {p:.4f})")
                    plt.xticks(rotation=45)
                    plt.show()

    return significant_columns
